# <u><h><center> Pauta Taller 1b</center></h></u>
## IIC2115 - Programación Como Herramienta para la Ingeniería
### Ayudante: Felipe Fuentes

In [2]:
import random #para cuando algo sea aleatorio, en este caso para generar mapas randoms
from math import log

## <u>Creación Mapa/Árbol</u>

In [3]:
class Arbol:
    # Creamos la estructura básica del árbol. Este representara el mapa de todas las habitaciones,
    # nos muestra como se conectan entre ellas. Los nodos hijos pueden ser guardados en alguna 
    # estructura como listas o diccionarios. Sin pérdidad de generalidad, en este ejemplo los
    # nodos hijos serán guardados en un diccionario.
    
    
    # Como estamos tratando con un árbol, las habitaciones las trataremos como nodos del árbol. 
    
    def __init__(self, id_habitacion, salida=False, id_padre=None, puerta= None):
        self.id_nodo= id_habitacion
        self.id_padre = id_padre #habitación anterior
        self.salida = salida #Boleano que nos avisa si estamos en una salida (si es que salimos es True)
        self.puerta = puerta
        self.hijos = {}
        self.n_hijos = 0
    
    def agregar_nodo(self, id_nodo, salida=False, id_padre=None):
        # Cada vez que agregamos un nodo verificamos primero si corresponde al nodo padre 
        # donde queremos agregar el nuevo nodo. Si no es el nodo, buscamos recursivamente 
        # a través de todos los nodos existentes hasta que encontremos el nodo correspondiente.
        
        if self.id_nodo == id_padre:
            # Si el nodo es el nodo padre, entonces actualizamos el diccionario 
            # con los hijos
            self.n_hijos+= 1
            self.hijos.update({self.n_hijos: Arbol(id_nodo, salida, id_padre, self.n_hijos)})
            return True
        else:
            # Si no, recursivamente seguimos buscando en el árbol el nodo padre
            
            for hijo in self.hijos.values():
                hijo.agregar_nodo(id_nodo, salida, id_padre)
        return False
                
    def obtener_nodo(self, id_nodo): #Función para obtener el nodo/habitación del id ingresado
        # recursivamente obtenemos el nodo siempre y cuando exista la posicion.
        
        if self.id_nodo == id_nodo:
            return self
        else:
            for hijo in self.hijos.values():
                nodo = hijo.obtener_nodo(id_nodo)
                
                if nodo:
                    # retorna el nodo si es que existe en el árbol
                    return nodo

## Recorrer Arbol (Encontrar caminos)

#### Para encontrar la salida, debemos recorrer el arbol de tal forma de encontrar el camino que nos lleve a la salida de la mansión embrujada.

### BFS

In [4]:
from collections import deque

class ArbolBFS(Arbol):
    # Heredamos de la clase original del Arbol y modificamos el metodo recorrer_arbol para usar 
    # el Breadth-first Search
    
    def __repr__(self):
        
        def recorrer_arbol(raiz):
            
            # Utiliamos una cola para almacenar los nodos por visitar
            # Se utiliza deque para el manejo de colas.
            Q = deque()
            Q.append(raiz)
            
            # Utilizamos una lista para registrar los nodos visitados
            visitados = []
            
            while len(Q) > 0:                
                p = Q.popleft()
                
                if p.id_nodo not in visitados:
                    
                    # Revisamos si el nodo ha sido visitado. Si no ha sido visitado
                    # lo agregamos a la lista de visitados
                    
                    visitados.append(p.id_nodo)

                    #visitamos el nodo
                    self.ret += "Llego por la puerta-> {}, habitación: {}, id_padre: {} -> salida: {}\n".format(p.puerta,
                        p.id_nodo, p.id_padre, p.salida)
                
                    # Agregamos todos los nodos hijos a la cola por visitar
                    for hijo in p.hijos.values():
                        Q.append(hijo)
                    
            return self

        self.ret = ''
        recorrer_arbol(self)
        return self.ret

### DFS

In [9]:
from collections import deque

class ArbolDFS(Arbol):
    
    # Heredamos de la clase original del Arbol y modificamos el metodo recorrer_arbol para usar 
    # el Depth-first Search (DFS)
    
    def __repr__(self):
        
        def recorrer_arbol(raiz):
            
            # En DPS utilizamos un stack para almacenar los nodos por visitar          
            Q = deque()
            Q.append(raiz)
            
            # Utilizaremos una lista para marcar los nodos visitados
            visitados = []

            while len(Q) > 0:                
                p = Q.pop()
                
                if p.id_nodo not in visitados:
                    
                    # Revisamos si el nodo ha sido visitado. Si no ha sido visitado
                    # lo agregamos a la lista de visitados
                    
                    visitados.append(p.id_nodo)
                    
                    self.ret += "Llego por la puerta-> {}, habitación: {}, id_padre: {} -> salida: {}\n".format(p.puerta,
                        p.id_nodo, p.id_padre, p.salida)

                    for hijo in p.hijos.values():
                        Q.append(hijo)

            return self
        
        self.ret = ''
        recorrer_arbol(self)
        
        return self.ret
    
    
    def encontrar_camino(self, mapa={}):
        
        self.ruta=''
        # En DPS utilizamos un stack para almacenar los nodos por visitar
        # Se utiliza deque para el manejo de colas.(en formato stack como se menciona)
        Q = deque()
        Q.append(self)

        # Utilizaremos una lista para marcar los nodos visitados
        visitados = []
        dicc_recorrido= {}
        while len(Q) > 0:                
            p = Q.pop()

            if p.id_nodo not in visitados:

                # Revisamos si el nodo ha sido visitado. Si no ha sido visitado
                # lo agregamos a la lista de visitados

                visitados.append(p.id_nodo)
                
                # registramos el "camino" del mapa que se esta recorriendo desde el inicio (habitacion 0)
                
                if p.id_padre==None:
                    dicc_recorrido["inicio"]= p
                else: 
                    dicc_recorrido[p.id_padre]= p
                
                #si es que la habitación esta en el mapa, entonces lo usamos para seleccionar la habitación correcta
                # y la agregamos a la cola sin necesidad de agregar las otras habitaciones de las otras puertas.
                
                if p.id_nodo in mapa:
                    Q.append(p.hijos[mapa[p.id_nodo]]) 
                else:
                    #si no esta en el mapa, se deben agregar todas las habitaciones detras de las puertas a la cola
                    #para ver posibles caminos.
                    
                    for hijo in p.hijos.values():
                        Q.append(hijo)
                    
                    
                if len(p.hijos.keys())==0:
                    #revisamos si no tiene hijos
                    #si no lo tienes llegamos al final de un camino
                    
                    if p.salida==True:
                        #si es una salida, entonces el camino nos sirve para escapar
                        break 
                    
                    #si no era un salida, se debe seguir buscando.
                
        nodo="inicio"
        while True:
            if dicc_recorrido[nodo].id_nodo in dicc_recorrido.keys():
                p= dicc_recorrido[nodo]
                self.ruta += "Llego por la puerta-> {}, habitación: {}, id_padre: {} -> salida: {}\n".format(p.puerta,
                 p.id_nodo, p.id_padre, p.salida)
                nodo= p.id_nodo
            else: 
                p= dicc_recorrido[nodo]
                self.ruta += "Llego por la puerta-> {}, habitación: {}, id_padre: {} -> salida: {}\n".format(p.puerta,
                 p.id_nodo, p.id_padre, p.salida)
                break
        return self.ruta

## Generar Mapa

In [12]:
cantidad_salidas= 2
habitaciones= 2**cantidad_salidas + cantidad_salidas

#Se puede usar tanto BFS y DFS para recorrer el mapa y encontrar una salida. En este caso, el ArbolBFS muestra
#un mapa de todas las habitaciones, mientras que ArbolDFS muestra una salida posible. El ArbolDFS se modifico para este caso,
#si desean ver como funciona DFS sin las modificaciones realizadas para este ejercicio, lean los contenidos de la semana.

MapaBFS = ArbolBFS(0, False) #muestra el mapa todal
MapaDFS = ArbolDFS(0, False) #imprime una salida 

diccionario_habitaciones= {}  #creamos un diccionario para contar que no sean más de 5 puertas.
diccionario_habitaciones[0]= 0 #habitación inicial de partida


while habitaciones>0:
    
    #creamos una habitación al azar, con id random
    id_habitacion= random.randint(0, 2**cantidad_salidas + cantidad_salidas) 
    
    #revisamos si ya se creo la habitación con este id
    #en caso de ya existir, se busca otro id.
    if id_habitacion not in diccionario_habitaciones: 
        
        diccionario_habitaciones[id_habitacion]= 0 

        
        while True:
            #seleccionamos un id de una habitación, sin importar si ya fue creada.
            id_padre= random.randint(0, 2**cantidad_salidas +  cantidad_salidas) 
            
            #revisamos si ya fue creada y que sea distinta a la habitación nueva
            
            if id_padre in diccionario_habitaciones and id_padre!= id_habitacion: 
                
                #revisamos que tenga menos de 5 puertas la habitación "padre"
            
                if diccionario_habitaciones[id_padre]<=5: 
                    
                    #Como se cumplen los criterios, agregamos la habitación nueva al mapa en 
                    #una puerta de la habitación padre.
                    
                    MapaBFS.agregar_nodo(id_habitacion, False, id_padre)
                    MapaDFS.agregar_nodo(id_habitacion, False, id_padre)
                    diccionario_habitaciones[id_padre]+= 1
                    
                    break
                else:
                    continue 
        #tenemos una habitación menos que crear
        habitaciones-=1 

#Agregamos las salidas

nodos_salida=[] #se utilizara para formar el mapa_parcial
contador=0
for habitacion in diccionario_habitaciones:
    
    #Buscamos en el diccionario de las habitaciones las que tengan 0 habitaciones "hijas"
    #Esto significa que estan al final de un camino/rama y por tanto pueden ser salidas.
    
    if diccionario_habitaciones[habitacion]==0:
        nodo_bfs= MapaBFS.obtener_nodo(habitacion)
        nodo_dfs= MapaDFS.obtener_nodo(habitacion)
        
        nodo_bfs.salida= True
        nodo_dfs.salida= True
        
        
        nodos_salida.append(habitacion)
        contador+=1
    if contador==cantidad_salidas:
        break        
        
print("MAPA DE TODAS LAS HABITACIONES USANDO BFS")
print(MapaBFS)
    
print("MAPA DE TODAS LAS HABITACIONES USANDO DFS")
print(MapaDFS)

MAPA DE TODAS LAS HABITACIONES USANDO BFS
Llego por la puerta-> None, habitación: 0, id_padre: None -> salida: False
Llego por la puerta-> 1, habitación: 5, id_padre: 0 -> salida: False
Llego por la puerta-> 2, habitación: 2, id_padre: 0 -> salida: True
Llego por la puerta-> 1, habitación: 6, id_padre: 5 -> salida: False
Llego por la puerta-> 1, habitación: 3, id_padre: 6 -> salida: False
Llego por la puerta-> 2, habitación: 1, id_padre: 6 -> salida: False
Llego por la puerta-> 1, habitación: 4, id_padre: 3 -> salida: True

MAPA DE TODAS LAS HABITACIONES USANDO DFS
Llego por la puerta-> None, habitación: 0, id_padre: None -> salida: False
Llego por la puerta-> 2, habitación: 2, id_padre: 0 -> salida: True
Llego por la puerta-> 1, habitación: 5, id_padre: 0 -> salida: False
Llego por la puerta-> 1, habitación: 6, id_padre: 5 -> salida: False
Llego por la puerta-> 2, habitación: 1, id_padre: 6 -> salida: False
Llego por la puerta-> 1, habitación: 3, id_padre: 6 -> salida: False
Llego por

### Mapa Parcial

In [13]:
#hacemos un diccionario de mapa parcial en donde las keys son los id de las habitaciones
#y los values son las puertas que se deben tomar.

mapa_parcial={}

habitaciones= 2**cantidad_salidas + cantidad_salidas
mapa_p_contador= 0

contador= 0

#nos aseguramos de que se cumpla la restriccion del enunciado y que termine la iteración

while log(habitaciones)-1>mapa_p_contador and 10000>contador:
    
    #como registramos los nodos de salida
    #podemos ver las habitaciones anteriores (padres) y por tanto podemos recorrer los caminos de la salida al inicio.
    #a continuación, recorremos el camino multiples veces con una probabilidad de agregar una habitación al mapa.
    
    for salida_id in nodos_salida:
        salida= MapaDFS.obtener_nodo(salida_id)
        habitacion= salida
        while habitacion.id_padre !=None:
            #hacemos una probabilidad de que este dentro del mapa parcial la habitacion
            
            prob= random.random()
            
            
            if prob>0.5 and habitacion.id_padre not in mapa_parcial:
                
                #agregamos la habitacion con la puerta correcta en el mapa
                
                mapa_parcial[habitacion.id_padre]= habitacion.puerta
                mapa_p_contador+=1
        
            habitacion= MapaDFS.obtener_nodo(habitacion.id_padre)
            
    contador+=1 

## Ejecución

In [18]:
print("MAPA PARCIAL ENCONTRADO")

mapa_parcial[0]=1
print(f"{mapa_parcial}\n")

print("RUTA POSIBLE PARA ESCAPAR")
print(MapaDFS.encontrar_camino())

MAPA PARCIAL ENCONTRADO
{6: 1, 0: 1}

RUTA POSIBLE PARA ESCAPAR
Llego por la puerta-> None, habitación: 0, id_padre: None -> salida: False
Llego por la puerta-> 2, habitación: 2, id_padre: 0 -> salida: True

